In [13]:
from boson_data_lib import extract_rho, extract_time
import numpy as np
import scipy.linalg as sl
import os
import pandas as pd
import glob
import h5py

In [2]:
#data_dir = "../DATA/"
data_dir = "../DATA/NO_CUT/"

tests_dir ="../TESTS/"

In [3]:
def fidelity_distances(data_directory, best_rho):
    
    init_rho = []
    fidelity_distances = []
    
    for i in range(0,20):
        gamma = '2.5133'
                
        file = data_directory + "State_D" + str(i+1) + '_CUT_data.h5' #'_2CUT_data.h5'
        rho, dt = extract_rho(file, gamma)
        
        init_rho.append(rho[1])
        
        dF = abs(np.trace(sl.sqrtm(sl.sqrtm(init_rho[i]) @ best_rho @ sl.sqrtm(init_rho[i]))))
        
        fidelity_distances.append(dF)
        
    return fidelity_distances

In [6]:
best_init_rho =  np.array([[ 0.96693975+0.j        , -0.17832301+0.00560344j],
                               [-0.17832301-0.00560344j,  0.03306025+0.j        ]])

fid_dists = fidelity_distances(data_dir, best_init_rho);

In [7]:
list_of_files = glob.glob(tests_dir+"*.h5") # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

../TESTS/DMD_TS_POP_RND_testD20_2024-Jan-31_at_16-18.h5


In [8]:
#latest_file = "DMD_TS_POP_testD20_2024-Jan-03_at_22-21.h5"
#latest_file = "DMD_TS_POP_testD20_2024-Jan-05_at_13-24.h5"

latest_file = "DMD_TS_POP_RND_testD20_2024-Jan-31_at_16-18.h5"

In [14]:
def read_DMD_TS_as_dataframe(filename, fid_distances, data_dir, tests_dir):

    df = pd.DataFrame()
    
    print("Processing...")
    
    with h5py.File(tests_dir+filename, "r") as f:
        
        #gammas = ['0.079477', '0.25133', '0.79477', '2.5133', '7.9477', '25.133', '79.477', '251.33']
        gammas = f.keys()
        
        for gamma in gammas:
            
            print(gamma)
            
            g = gamma[6:]
            
            init_states = f[gamma].keys()
            
            for state in init_states:
                
                fidelity = f[gamma][state]["F_sid_lme"][()]
                
                ser_len = len(fidelity)

                gamma_column = [g] * ser_len
                state_column = [state[7:]] * ser_len

                fid_dist_column = [fid_distances[int(state[7:])-1]] * ser_len      

                f_name = data_dir + "/" + state + "_CUT_data.h5" #"_2CUT_data.h5"
                #f_name = data_dir + "\\" + state + "_2CUT_data.h5"
                t, dt = extract_time(f_name, g)
                time_column = t
                gamma_time_column = np.array(t)*float(g) 

                d_ser = {'Gamma': gamma_column,
                         'State': state_column,
                         'Time': time_column, 
                         'gt': gamma_time_column,
                         'Fidelity': fidelity,
                         'Infidelity': 1-fidelity,
                         'Distance': fid_dist_column}

                df_ser = pd.DataFrame(data = d_ser)   
                df = pd.concat([df, df_ser])
    
    print(" done!")
    
    return df

In [15]:
df = read_DMD_TS_as_dataframe(latest_file, fid_dists, data_dir, tests_dir)

Processing...
gamma_0.079477
gamma_0.25133
gamma_0.79477
gamma_2.5133
gamma_25.133
gamma_251.33
gamma_7.9477
gamma_79.477
 done!


In [16]:
os.chdir("../TESTS/")

df.to_pickle("dataframe_"+latest_file+".pkl")

In [17]:
print("dataframe_"+latest_file+".pkl")

dataframe_DMD_TS_POP_RND_testD20_2024-Jan-31_at_16-18.h5.pkl
